# Ntpl 0 0 3_Lhcb Open Data
Convert LHCb run 1 open data from a TTree to RNTuple.
This tutorial illustrates data conversion for a simple, tabular data model.
For reading, the tutorial shows the use of an ntuple View, which selectively accesses specific fields.
If a view is used for reading, there is no need to define the data model as an RNTupleModel first.
The advantage of a view is that it directly accesses RNTuple's data buffers without making an additional
memory copy.




**Author:** The ROOT Team  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 04:05 AM.</small></i>

Note: the rntuple classes are experimental at this point.
 Functionality, interface, and data format is still subject to changes.
 Do not use for real data!

In [1]:
%%cpp -d
#include <ROOT/RField.hxx>
#include <ROOT/RNTuple.hxx>
#include <ROOT/RNTupleModel.hxx>

#include <TBranch.h>
#include <TCanvas.h>
#include <TFile.h>
#include <TH1F.h>
#include <TLeaf.h>
#include <TSystem.h>
#include <TTree.h>

#include <cassert>
#include <memory>
#include <vector>

input_line_26:1:10: fatal error: 'ROOT/RField.hxx' file not found
#include <ROOT/RField.hxx>
         ^~~~~~~~~~~~~~~~~


Import classes from experimental namespace for the time being

In [2]:
using RNTupleModel = ROOT::Experimental::RNTupleModel;
using RFieldBase = ROOT::Experimental::Detail::RFieldBase;
using RNTupleReader = ROOT::Experimental::RNTupleReader;
using RNTupleWriter = ROOT::Experimental::RNTupleWriter;

constexpr char const* kTreeFileName = "http://root.cern.ch/files/LHCb/lhcb_B2HHH_MagnetUp.root";
constexpr char const* kNTupleFileName = "ntpl003_lhcbOpenData.root";

input_line_27:1:42: error: no type named 'RNTupleModel' in namespace 'ROOT::Experimental'
using RNTupleModel = ROOT::Experimental::RNTupleModel;void __cling_Un1Qu32(void* vpClingValue) {
                     ~~~~~~~~~~~~~~~~~~~~^
input_line_27:3:40: error: 'Detail' is not a class, namespace, or enumeration
using RFieldBase = ROOT::Experimental::Detail::RFieldBase;
                                       ^
/mnt/build/workspace/root-makedoc-v618/rootspi/rdoc/src/v6-18-00-patches.build/include/ROOT/RNotFn.hxx:31:11: note: 'Detail' declared here
namespace Detail {
          ^
input_line_27:4:43: error: no type named 'RNTupleReader' in namespace 'ROOT::Experimental'
using RNTupleReader = ROOT::Experimental::RNTupleReader;
                      ~~~~~~~~~~~~~~~~~~~~^
input_line_27:5:43: error: no type named 'RNTupleWriter' in namespace 'ROOT::Experimental'
using RNTupleWriter = ROOT::Experimental::RNTupleWriter;
                      ~~~~~~~~~~~~~~~~~~~~^


 A helper function is created: 

In [3]:
%%cpp -d
void Convert() {
   std::unique_ptr<TFile> f(TFile::Open(kTreeFileName));
   assert(f && ! f->IsZombie());

   // Get a unique pointer to an empty RNTuple model
   auto model = RNTupleModel::Create();

   // We create RNTuple fields based on the types found in the TTree
   // This simple approach only works for trees with simple branches and only one leaf per branch
   auto tree = f->Get<TTree>("DecayTree");
   for (auto b : TRangeDynCast<TBranch>(*tree->GetListOfBranches())) {
      // The dynamic cast to TBranch should never fail for GetListOfBranches()
      assert(b);

      // We assume every branch has a single leaf
      TLeaf *l = static_cast<TLeaf*>(b->GetListOfLeaves()->First());

      // Create an ntuple field with the same name and type than the tree branch
      auto field = RFieldBase::Create(l->GetName(), l->GetTypeName());
      std::cout << "Convert leaf " << l->GetName() << " [" << l->GetTypeName() << "]"
                << " --> " << "field " << field->GetName() << " [" << field->GetType() << "]" << std::endl;

      // Hand over ownership of the field to the ntuple model.  This will also create a memory location attached
      // to the model's default entry, that will be used to place the data supposed to be written
      model->AddField(std::unique_ptr<RFieldBase>(field));

      // We connect the model's default entry's memory location for the new field to the branch, so that we can
      // fill the ntuple with the data read from the TTree
      void *fieldDataPtr = model->GetDefaultEntry()->GetValue(l->GetName()).GetRawPtr();
      tree->SetBranchAddress(b->GetName(), fieldDataPtr);
   }

   // The new ntuple takes ownership of the model
   auto ntuple = RNTupleWriter::Recreate(std::move(model), "DecayTree", kNTupleFileName);

   auto nEntries = tree->GetEntries();
   for (decltype(nEntries) i = 0; i < nEntries; ++i) {
      tree->GetEntry(i);
      ntuple->Fill();

      if (i && i % 100000 == 0)
         std::cout << "Wrote " << i << " entries" << std::endl;
   }
}

input_line_32:2:41: error: use of undeclared identifier 'kTreeFileName'
   std::unique_ptr<TFile> f(TFile::Open(kTreeFileName));
                                        ^
input_line_32:6:17: error: use of undeclared identifier 'RNTupleModel'
   auto model = RNTupleModel::Create();
                ^
input_line_32:19:20: error: use of undeclared identifier 'RFieldBase'
      auto field = RFieldBase::Create(l->GetName(), l->GetTypeName());
                   ^
input_line_32:25:39: error: use of undeclared identifier 'RFieldBase'
      model->AddField(std::unique_ptr<RFieldBase>(field));
                                      ^
input_line_32:34:18: error: use of undeclared identifier 'RNTupleWriter'
   auto ntuple = RNTupleWriter::Recreate(std::move(model), "DecayTree", kNTupleFileName);
                 ^
input_line_32:34:73: error: use of undeclared identifier 'kNTupleFileName'
   auto ntuple = RNTupleWriter::Recreate(std::move(model), "DecayTree", kNTupleFileName);
                      

In [4]:
if (gSystem->AccessPathName(kNTupleFileName))
   Convert();

input_line_34:2:57: error: use of undeclared identifier 'kNTupleFileName'
 (((*(class TSystem **)0x7f851a826af0))->AccessPathName(kNTupleFileName))
                                                        ^
Error in <HandleInterpreterException>: Error evaluating expression (((*(class TSystem **)0x7f851a826af0))->AccessPathName(kNTupleFileName)).
Execution of your code was aborted.


Create histogram of the flight distance

We open the ntuple without specifiying an explicit model first, but instead use a view on the field we are
 interested in

In [5]:
auto ntuple = RNTupleReader::Open("DecayTree", kNTupleFileName);

input_line_35:2:16: error: 'RNTupleReader' is not a class, namespace, or enumeration
 auto ntuple = RNTupleReader::Open("DecayTree", kNTupleFileName);
               ^
input_line_35:2:16: note: 'RNTupleReader' declared here
input_line_35:2:49: error: use of undeclared identifier 'kNTupleFileName'
 auto ntuple = RNTupleReader::Open("DecayTree", kNTupleFileName);
                                                ^


The view wraps a read-only double value and accesses directly the ntuple's data buffers

In [6]:
auto viewFlightDistance = ntuple->GetView<double>("B_FlightDistance");

auto c = new TCanvas("c", "B Flight Distance", 200, 10, 700, 500);
TH1F h("h", "B Flight Distance", 200, 0, 140);
h.SetFillColor(48);

for (auto i : ntuple->GetViewRange()) {
   // Note that we do not load an entry in this loop, i.e. we avoid the memory copy of loading the data into
   // the memory location given by the entry
   h.Fill(viewFlightDistance(i));
}

h.DrawCopy();

input_line_36:2:36: error: use 'template' keyword to treat 'GetView' as a dependent template name
 auto viewFlightDistance = ntuple->GetView<double>("B_FlightDistance");
                                   ^
                                   template 
input_line_36:8:15: error: use of undeclared identifier 'ntuple'
for (auto i : ntuple->GetViewRange()) {
              ^
input_line_36:2:2: error: Syntax error
 auto viewFlightDistance = ntuple->GetView<double>("B_FlightDistance");
 ^
FunctionDecl 0x55c6c73f57a0 <input_line_36:1:1, line:16:1> line:1:6 __cling_Un1Qu36 'void (void *)'
|-ParmVarDecl 0x55c6c73f56f8 <col:22, col:28> col:28 vpClingValue 'void *'
|-CompoundStmt 0x55c6c74007a0 <col:42, line:16:1>
| |-DeclStmt 0x55c6c73f5bd0 <line:2:2, col:71>
| | `-VarDecl 0x55c6c73f5870 <col:2, col:70> col:7 used viewFlightDistance 'auto' cinit
| |   `-CallExpr 0x55c6c73f5b78 <col:28, col:70> '<dependent type>'
| |     |-CXXDependentScopeMemberExpr 0x55c6c73f5aa8 <col:28, col:50> '<dependent typ

Draw all canvases 

In [7]:
gROOT->GetListOfCanvases()->Draw()